In [1]:
import os
import os.path
from urllib.request import urlopen
from requests import get
from bs4 import BeautifulSoup
from collections import defaultdict

from pandas import DataFrame
import PIL.Image
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn import cluster
from sklearn.cluster import KMeans

base_url = 'http://www.libertylondon.com'
url = 'http://www.libertylondon.com/uk/department/fabric/'
start_urls = ['/uk/department/fabric/cotton-tana-lawn/',
             '/uk/department/fabric/silk/',
             '/uk/department/fabric/wool/',
             '/uk/department/fabric/linen/',
             '/uk/department/fabric/poplin/',
             '/uk/department/fabric/viscose-jersey/',
             '/uk/department/fabric/cord/',
             '/uk/department/fabric/fleece/',
             '/uk/department/fabric/cotton-wool/',
             '/uk/department/fabric/furnishing-fabric/',
             '/uk/department/sale/fabric/50-off/',
             '/uk/edits/classics/']

In [28]:
def extract_fabrics(soup):
    fabrics = []
    for product in soup.find_all(class_="product product-tile"):
        imgs = product.find_all('img')
        sales = product.find(class_='price-sales')
        amount = ' '.join(product.find(class_='price-sales').text.split())
        main_link = product.find(class_='thumb-link')['href']
        for img in imgs:
            title, src = img['title'], img['data-src']
            fabrics += [[title, src, amount, base_url + main_link]]
    return fabrics

In [16]:
def extract_images(url, title):
    images = []
    r = get(url)
    soup = BeautifulSoup(r.text, "lxml")
    image_links = soup.find_all('img')
    for im in image_links:
        name = im.get('alt', '')
        if name == title:
            src = im['src']
            images += [im['src']]
            try:
                #download_image(src)
            except Exception as e:
                print('Issue', e, images, title, url)
    return images

In [4]:
def download_image(link):
    r = get(link, stream=True)
    outfile = link.split('/')[-1].split('$')[0].replace('?', '')
    im = PIL.Image.open(r.raw)
    if 'jpg' not in outfile:
        outfile += '.jpg'
    im.save('data/2019/' + outfile)

In [5]:
def kmeans_colour(im):
    ima = np.asarray(im)[:,:,:3]
    w,h,d = ima.shape
    X = np.reshape(ima, (w*h, d))
    kcluster = cluster.KMeans(init='k-means++', n_clusters=16)
    kcluster.fit(X)
    predictions = kcluster.predict(X)
    clusters = kcluster.cluster_centers_
    image = np.zeros((w, h, 3))
    predictions_idx = 0
    colour_distribution = defaultdict(int)
    for i in range(w):
        for j in range(h):
            cluster_int = predictions[predictions_idx]
            image[i][j] = clusters[cluster_int]
            predictions_idx += 1
            colour_distribution[cluster_int] += 1
    for k, v in colour_distribution.items():
        v = v / (w*h)
        colour_distribution[k] = v
    im = PIL.Image.fromarray(image.astype('uint8'), 'RGB')
    return im, colour_distribution, clusters

In [6]:
def amount_of_white(colour_distribution, clusters):
    total = 0
    for index, c in enumerate(clusters):
        if sum(list(c)) > 750:
            total += colour_distribution[index]
    return total

In [7]:
def crop_images():
    rootdir = 'data/2019/'
    data = []
    for _, _, files in os.walk(rootdir):
        for file in files:
            if 'jpg' in file:
                if not os.path.isfile('data/2019/cropped/' + file): 
                    img = rootdir + file
                    im = PIL.Image.open(img)
                    kim, colour_dis, clu = kmeans_colour(im)
                    if amount_of_white(colour_dis, clu) > 0.20:
                        print('greater', file)
                        im = im.crop((70,120,600,650))
                        kim = kim.crop((70,120,600,650))
                    im.save('data/cropped/' + file)
                    kim.save('data/color/' + file)
                    data += [[file, colour_dis, clu]] 
    return data

In [31]:
fabrics = []
for u in start_urls:
    print(u)
    u = base_url + u
    r = get(u + '?sz=120#sz=120&start=120')
    print(r)
    soup = BeautifulSoup(r.text, "lxml")
    more_fabrics = extract_fabrics(soup)
    print(len(more_fabrics))
    for f in more_fabrics:
        f += [u]
    fabrics += more_fabrics
data = DataFrame(fabrics, columns=['title', 'img_src', 'amount', 'url', 'from_url'])
data['images'] = data.apply(lambda x: extract_images(x['url'], x['title']), axis=1)
data.to_csv('liberty_prints.csv')

/uk/department/fabric/cotton-tana-lawn/
<Response [200]>
240


In [296]:
data2 = crop_images()

/Users/Ruby/Envs/data-sci3/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype uint8 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


greater 000515498-R047925006-2.jpg


/Users/Ruby/Envs/data-sci3/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype uint8 was converted to float64.
  warnings.warn(msg, DataConversionWarning)
/Users/Ruby/Envs/data-sci3/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype uint8 was converted to float64.
  warnings.warn(msg, DataConversionWarning)
/Users/Ruby/Envs/data-sci3/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype uint8 was converted to float64.
  warnings.warn(msg, DataConversionWarning)
/Users/Ruby/Envs/data-sci3/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype uint8 was converted to float64.
  warnings.warn(msg, DataConversionWarning)
/Users/Ruby/Envs/data-sci3/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype uint8 was converted to float64.

greater 000515511-R047941006-4.jpg


/Users/Ruby/Envs/data-sci3/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype uint8 was converted to float64.
  warnings.warn(msg, DataConversionWarning)
/Users/Ruby/Envs/data-sci3/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype uint8 was converted to float64.
  warnings.warn(msg, DataConversionWarning)
/Users/Ruby/Envs/data-sci3/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype uint8 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


...

greater 80680-1001442591-2.jpg


/Users/Ruby/Envs/data-sci3/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype uint8 was converted to float64.
  warnings.warn(msg, DataConversionWarning)
/Users/Ruby/Envs/data-sci3/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype uint8 was converted to float64.
  warnings.warn(msg, DataConversionWarning)
/Users/Ruby/Envs/data-sci3/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype uint8 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


greater 80680-1001442591-5.jpg


/Users/Ruby/Envs/data-sci3/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype uint8 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


greater 000515498-R047925006-2.jpg


In [297]:
data_colours = DataFrame(data, columns=['image_name', 'colour_distribution', 'clusters'])
data_colours.to_csv('colour_dis.csv')